In [1]:
debugging = True
IPTS = 19558

# Description 

# Python Import

In [2]:
from __code.__all import custom_style
custom_style.style()

In [3]:
import matplotlib.patches as patches
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import os

from NeuNorm.normalization import Normalization
from NeuNorm.roi import ROI

from __code import utilities, gui_widgets, file_handler
import ipywe.fileselector

from ipywidgets.widgets import interact
from ipywidgets import widgets
from IPython.core.display import display, HTML

if debugging:
    ipts = IPTS
else:
    ipts = utilities.get_ipts()
working_dir = utilities.get_working_dir(ipts=ipts, debugging=debugging)
print("Working dir: {}".format(working_dir))

Working dir: /Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19558


# Select Sample Images 

In [4]:
list_images_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select samples images', start_dir=working_dir, multiple=True)
list_images_ui.show()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

# Select OB 

In [6]:
list_ob_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select OB images', start_dir=working_dir, multiple=True)
list_ob_ui.show()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

# Loding data 

In [7]:
o_norm = Normalization()

# sample
list_images = list_images_ui.selected
o_norm.load(file=list_images, notebook=True)
sample_array = o_norm.data['sample']['data']

# ob
list_obs = list_ob_ui.selected
o_norm.load(file=list_obs, data_type='ob', notebook=True)
ob_array = o_norm.data['ob']['data']

A Jupyter Widget

A Jupyter Widget

# Preview Data

In [10]:
def plot_images(index):
    fig = plt.figure(figsize=(5,5))
    ax_img = plt.subplot(111)
    ax_img.imshow(sample_array[index], cmap='viridis')
    
preview = interact(plot_images,
                  index = widgets.IntSlider(min=0, 
                                             max=len(list_images)-1,
                                                step=1,
                                                value=0,
                                                description='Image Index',
                                           continuous_update=False))

A Jupyter Widget

In [11]:
def plot_obs(index):
    fig = plt.figure(figsize=(5,5))
    ax_img = plt.subplot(111)
    ax_img.imshow(ob_array[index], cmap='viridis')
    
preview = interact(plot_obs,
                  index = widgets.IntSlider(min=0, 
                                             max=len(list_obs)-1,
                                                step=1,
                                                value=0,
                                                description='Image Index',
                                           continuous_update=False))

A Jupyter Widget

# Select Background Region 

In [21]:
integrated_array = np.array([_array for _array in sample_array])
mean_array = integrated_array.mean(axis=0)

[height, width] = np.shape(sample_array[0])

def plot_roi(x_left, y_top, width, height):
    
    fig = plt.figure(figsize=(5,5))
    ax_img = plt.subplot(111)
    ax_img.imshow(mean_array,
                 cmap='viridis')
    
    _rectangle = patches.Rectangle((x_left, y_top),
                                  width,
                                  height,
                                  edgecolor = 'white',
                                  linewidth = 2,
                                  fill = False)
    ax_img.add_patch(_rectangle)
    
    return [x_left, y_top, width, height]
    
preview_bragg_edge = interact(plot_roi,
                  x_left = widgets.IntSlider(min=0,
                                            max=width,
                                            step=1,
                                            value=8,
                                            description='X Left',
                                            continuous_update=False),
                  y_top = widgets.IntSlider(min=0,
                                           max=height,
                                           value=13,
                                           step=1,
                                           description='Y Top',
                                           continuous_update=False),
                  width = widgets.IntSlider(min=0,
                                           max=width,
                                           step=1,
                                           value=60,
                                           description="Width",
                                           continuous_update=False),
                  height = widgets.IntSlider(min=0,
                                            max=height,
                                            step=1,
                                            value=100,
                                            description='Height',
                                            continuous_update=False))

# Normalization

In [26]:
[x_left, y_top, width_roi, height_roi] = preview_bragg_edge.widget.result

_roi = ROI(x0=x_left, y0=y_top, width=width_roi, height=height_roi)
o_norm.normalization(roi = _roi, notebook=True)

normalized_data_array = o_norm.get_normalized_data()

A Jupyter Widget

# Export 

In [28]:
output_folder_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select Output Folder', start_dir=working_dir, multiple=False, type='directory')
output_folder_ui.show()

A Jupyter Widget

A Jupyter Widget

In [36]:
output_folder = os.path.abspath(os.path.join(output_folder_ui.selected, 'normalization'))
utilities.make_dir(dir=output_folder)

w = widgets.IntProgress()
w.max = len(list_images)
display(w)

for _index, _file in enumerate(list_images):
    basename = os.path.basename(_file)
    _base, _ext = os.path.splitext(basename)
    output_file_name = os.path.join(output_folder, _base + '.tiff')
    file_handler.make_tiff(filename=output_file_name, data=normalized_data_array[_index])
    
    w.value = _index+1

A Jupyter Widget